# Setting Up Your Local SQL Learning Environment with DuckDB 🚀

This notebook will create a local DuckDB database with sample data that we'll use throughout the course.
You only need to run this once to set up your learning environment.

DuckDB is a high-performance analytical database system that's perfect for learning SQL!

In [ ]:
import duckdb
import os
import pandas as pd

# Create the data directory if it doesn't exist
os.makedirs('../data', exist_ok=True)

# Connect to a new DuckDB database
db_path = '../data/movies.duckdb'
conn = duckdb.connect(db_path)

# Enable foreign key constraints
conn.execute('PRAGMA foreign_keys = ON;')

## Creating the Database Schema 📝

Let's create all our tables with proper relationships:

In [ ]:
# Create tables
conn.execute("""
-- Movies table
CREATE TABLE movies (
    movie_id INTEGER PRIMARY KEY,
    title VARCHAR(255) NOT NULL,
    release_year INTEGER NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Genres table
CREATE TABLE genres (
    genre_id INTEGER PRIMARY KEY,
    name VARCHAR(50) NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Movie-Genre junction table
CREATE TABLE movie_genres (
    movie_id INTEGER NOT NULL,
    genre_id INTEGER NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (movie_id, genre_id),
    FOREIGN KEY (movie_id) REFERENCES movies(movie_id),
    FOREIGN KEY (genre_id) REFERENCES genres(genre_id)
);

-- Actors table
CREATE TABLE actors (
    actor_id INTEGER PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    birth_year INTEGER,
    bio TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Movie-Actor junction table
CREATE TABLE movie_actors (
    movie_id INTEGER NOT NULL,
    actor_id INTEGER NOT NULL,
    role_name VARCHAR(100),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (movie_id, actor_id),
    FOREIGN KEY (movie_id) REFERENCES movies(movie_id),
    FOREIGN KEY (actor_id) REFERENCES actors(actor_id)
);

-- Directors table
CREATE TABLE directors (
    director_id INTEGER PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    bio TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Movie-Director junction table
CREATE TABLE movie_directors (
    movie_id INTEGER NOT NULL,
    director_id INTEGER NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (movie_id, director_id),
    FOREIGN KEY (movie_id) REFERENCES movies(movie_id),
    FOREIGN KEY (director_id) REFERENCES directors(director_id)
);

-- Users table
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY,
    username VARCHAR(50) NOT NULL UNIQUE,
    email VARCHAR(255) NOT NULL UNIQUE,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Reviews table
CREATE TABLE reviews (
    review_id INTEGER PRIMARY KEY,
    user_id INTEGER NOT NULL,
    movie_id INTEGER NOT NULL,
    rating INTEGER NOT NULL CHECK (rating BETWEEN 1 AND 10),
    review_text TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES users(user_id),
    FOREIGN KEY (movie_id) REFERENCES movies(movie_id)
);
""")

## Adding Sample Data 📚

Let's populate our tables with some interesting movie data:

In [ ]:
# Insert sample data
conn.execute("""
-- Insert genres
INSERT INTO genres (genre_id, name) VALUES
    (1, 'Action'),
    (2, 'Comedy'),
    (3, 'Drama'),
    (4, 'Science Fiction'),
    (5, 'Animation');

-- Insert movies
INSERT INTO movies (movie_id, title, release_year) VALUES
    (1, 'The Matrix', 1999),
    (2, 'Inception', 2010),
    (3, 'Toy Story', 1995),
    (4, 'Pulp Fiction', 1994),
    (5, 'The Dark Knight', 2008);

-- Insert movie-genre relationships
INSERT INTO movie_genres (movie_id, genre_id) VALUES
    (1, 1), (1, 4),  -- The Matrix: Action, Sci-Fi
    (2, 1), (2, 4),  -- Inception: Action, Sci-Fi
    (3, 5), (3, 2),  -- Toy Story: Animation, Comedy
    (4, 3), (4, 2),  -- Pulp Fiction: Drama, Comedy
    (5, 1), (5, 3);  -- The Dark Knight: Action, Drama

-- Insert directors
INSERT INTO directors (director_id, name) VALUES
    (1, 'Christopher Nolan'),
    (2, 'Lana Wachowski'),
    (3, 'John Lasseter'),
    (4, 'Quentin Tarantino');

-- Insert movie-director relationships
INSERT INTO movie_directors (movie_id, director_id) VALUES
    (1, 2),  -- The Matrix - Wachowski
    (2, 1),  -- Inception - Nolan
    (3, 3),  -- Toy Story - Lasseter
    (4, 4),  -- Pulp Fiction - Tarantino
    (5, 1);  -- The Dark Knight - Nolan

-- Insert some users
INSERT INTO users (user_id, username, email) VALUES
    (1, 'moviefan', 'fan@movies.com'),
    (2, 'cinephile', 'cine@movies.com'),
    (3, 'reviewer', 'review@movies.com');

-- Insert some reviews
INSERT INTO reviews (review_id, user_id, movie_id, rating, review_text) VALUES
    (1, 1, 1, 9, 'Revolutionary sci-fi masterpiece!'),
    (2, 2, 1, 8, 'Mind-bending and innovative'),
    (3, 1, 2, 9, 'A dream within a dream'),
    (4, 3, 3, 10, 'Perfect family entertainment'),
    (5, 2, 4, 9, 'A cinematic masterpiece');
""")

## Testing Our Database 🧪

Let's run some test queries to make sure everything is working:

In [ ]:
def test_query(query, description):
    print(f"\n{description}:")
    return conn.execute(query).df()

# Test basic movie information
print(test_query("""
    SELECT m.title, m.release_year, g.name as genre
    FROM movies m
    JOIN movie_genres mg ON m.movie_id = mg.movie_id
    JOIN genres g ON mg.genre_id = g.genre_id
    ORDER BY m.title, g.name
""", "Movies and their genres"))

# Test movie reviews
print(test_query("""
    SELECT m.title, 
           ROUND(AVG(r.rating), 2) as avg_rating,
           COUNT(r.review_id) as review_count
    FROM movies m
    LEFT JOIN reviews r ON m.movie_id = r.movie_id
    GROUP BY m.title
    ORDER BY avg_rating DESC
""", "Movie ratings summary"))

## 🎉 Success!

Your DuckDB database is now set up and ready for learning SQL! Here's what we've accomplished:

1. Created a new DuckDB database
2. Set up tables with proper relationships
3. Added sample movie data
4. Verified everything works with test queries

You can now move on to the learning notebooks. If you ever need to reset the database, just run this notebook again!

### Next Steps 🎯

1. Open `01_basic_queries.ipynb` to start learning SQL basics
2. Try writing your own queries using this database
3. Feel free to add more movies and reviews to practice with!